In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
!pwd

/home/aditya/MLOPS/E2E-Data-Science-Project


In [36]:
from pydantic import BaseModel, HttpUrl
from pathlib import Path

class DataIngestionConfig(BaseModel):
    root_dir: Path
    source_url: HttpUrl  # Ensures it's a valid URL
    local_datafile: Path
    unzip_dir: Path

In [37]:
from src.DataScienceWorkflow.constants import *
from src.DataScienceWorkflow.utils.common import read_yaml,create_directories
from box import Box

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:

        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_datafile = config.local_datafile,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [38]:
cm = ConfigurationManager()
data_ingestion_config = cm.get_data_ingestion_config()

data_ingestion_config

[2025-03-29 14:50:39,725: INFO: common : Yaml File: config/config.yaml loaded successfully]
[2025-03-29 14:50:39,726: INFO: common : Yaml File: params.yaml loaded successfully]
[2025-03-29 14:50:39,726: INFO: common : Yaml File: schema.yaml loaded successfully]
[2025-03-29 14:50:39,727: INFO: common : created directory at: artifacts]
[2025-03-29 14:50:39,728: INFO: common : created directory at: artifacts/data_ingestion]


DataIngestionConfig(root_dir=PosixPath('artifacts/data_ingestion'), source_url=HttpUrl('https://github.com/krishnaik06/datasets/raw/refs/heads/main/winequality-data.zip'), local_datafile=PosixPath('artifacts/data_ingestion/data.zip'), unzip_dir=PosixPath('artifacts/data_ingestion'))

In [39]:
from src.DataScienceWorkflow import logger
from urllib import request
import zipfile

class DataIngestion:

    def __init__(self,config):
        self.config = config
    
    def download_file(self):

        if not os.path.exists(self.config.local_datafile):
            filename,headers = request.urlretrieve(
                url = str(self.config.source_url),
                filename = self.config.local_datafile
            )
            logger.info(f"{filename} downloaded! with following info {headers}")
        else:
            logger.info(f"File already exists")
    
    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_datafile,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [40]:
di = DataIngestion(data_ingestion_config)

di.download_file()
di.extract_zip_file()

[2025-03-29 14:50:44,331: INFO: 1030460877 : artifacts/data_ingestion/data.zip downloaded! with following info Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6C94:2A8B2D:3C9AA8:99B324:67E7BACB
Accept-Ranges: bytes
Date: Sat, 29 Mar 2025 09:20:44 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100029-HYD
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1743240044.294735,VS0,VE0
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 1a6b0e7fa6b327a35696b5151a924d278d77c8a6
Expires: Sat, 29 Mar 2025 09:25:44 GMT
Source-Age: 156

]
